In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

## Creating a list of Supreme Court Opinions and their URLs
I found that caselaw has a repository of Supreme Court ruling opinions spanning back over 100 years. This notebook contains the code I wrote to compile the list of these URLs and titles to use to grab the opinions.

If you want the data I used for this project, you can start by running these cells. 

In [2]:
root_url = "http://caselaw.findlaw.com/court/us-supreme-court/years/"

In [24]:
years = [year for year in range(2016,2017)]
soupurls = []
souptitles = []
yearlist = []
for year in years:
    search = root_url + str(year)
    response = requests.get(search)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    souped = soup.findAll(class_ = "srpcaselawtr")
    souped = str(souped)
    #souped = souped.findAll("td", {"valign" : "top"})
    souplist = soup.findAll("a")
    for i in souplist:
        if re.search("us-supreme-court", str(i)) and not re.search("years", str(i)) and not re.search("/court/", str(i)): 
            yearlist.append(year)
            soupurls.append(i["href"])
            souptitles.append(i["title"])


In [25]:

souped = souped.split('<td valign="top">')


In [40]:
souped

['[<tr class="srpcaselawtr">\n',
 '<a href="http://caselaw.findlaw.com/us-supreme-court/15-584.html" text-transform:uppercase="" title="AMERICAN FREEDOM DEFENSE INITIATIVE, &lt;SPAN STYLE=">&gt;AMERICAN FREEDOM DEFENSE INITIATIVE, <span style="TEXT-TRANSFORM: UPPERCASE"><small>ET AL.</small></span> <em> v. </em> KING COUNTY, WASHINGTON</a></td>\n',
 'March 7, 2016</td>\n',
 'No. 15-584</td>\n</tr>, <tr class="srpcaselawtr altrow">\n',
 '<a href="http://caselaw.findlaw.com/us-supreme-court/14-1382.html" text-transform:uppercase="" title="AMERICOLD REALTY TRUST &lt;EM&gt;v&lt;/EM&gt;. CONAGRA FOODS, INC., &lt;SPAN STYLE=">&gt;AMERICOLD REALTY TRUST <em>v</em>. CONAGRA FOODS, INC., <span style="TEXT-TRANSFORM: UPPERCASE"><small>ET AL.</small></span></a></td>\n',
 'March 7, 2016</td>\n',
 'No. 14-1382</td>\n</tr>, <tr class="srpcaselawtr">\n',
 '<a href="http://caselaw.findlaw.com/us-supreme-court/15-278.html" text-transform:uppercase="" title="AMGEN INC., &lt;SPAN STYLE=">&gt;AMGEN INC., 

In [139]:
import re
def cleanup_results(text):
    text2 = []
    for i in text:
        if i[:22] == '[<tr class="srpcaselaw':
            continue
        if i[:2] == '<a':
                i = i.split('text-transform:uppercase="" title="')
                for j in i:
                    j = j.replace('<a ', "").replace("</small>", "").replace('&lt;SPAN STYLE=">&gt;', "") \
                    .replace('<span style="TEXT-TRANSFORM: UPPERCASE">', "").replace("\n", "").replace("<em>", "").replace("</em>","") \
                    .replace('tr class="srpcaselawtr altrow">', "").replace('<span style="TEXT-TRANSFORM: UPPERCASE"><small>', "") \
                    .replace('href="', "").replace("</small>", "").replace("</td>", "").replace('" ', "").replace("<small>", "") \
                    .replace("</span>", "").replace("</a>", "").replace("&lt;EM&gt;v&lt;/EM&gt;", "").replace("&lt;EM&gt; ", "") \
                    .replace("&lt;/EM&gt; ", "").replace("amp;AMP;", "").replace(">", "")
                    if re.search('title', j):
                        j = j.split('title="')
                        #text2.append(j)
                    text2.append(j)
        else:
            i = i.replace('<span style="TEXT-TRANSFORM: UPPERCASE">', "").replace("\n", "").replace("<em>", "").replace("</em>","") \
            .replace('tr class="srpcaselawtr altrow">', "").replace('<span style="TEXT-TRANSFORM: UPPERCASE"><small>', "") \
            .replace('href="', "").replace("</small>", "").replace("</td>", "").replace("</tr>", "") \
            .replace('<a ', "").replace("</small>", "").replace('&lt;SPAN STYLE=">&gt;', "").replace(', <tr class="srpcaselawtr">', "") \
            .replace(", <", "")
            text2.append(i)
    return text2

In [144]:
j = 'http:'
if re.search('http', j):
    print("match")

match


In [163]:
case_detail_list = cleanup_results(souped)

In [169]:
cases = {}
for i in case_detail_list:
    if type(i) == type(list()):
        j = i[1].split('"')
        j = j[0]
        cases[i[0]] = [j]
#     else:
#         cases[j].append(i)

print(cases)        
        

{'http://caselaw.findlaw.com/us-supreme-court/14-10078.html': ['JAIME CAETANO v. MASSACHUSETTS'], 'http://caselaw.findlaw.com/us-supreme-court/15-458.html': ['DIETZ . BOULDIN'], 'http://caselaw.findlaw.com/us-supreme-court/14-449.html': ['KANSAS . CARR'], 'http://caselaw.findlaw.com/us-supreme-court/14-8913.html': ['MOLINA-MARTINEZ . UNITED STATES'], 'http://caselaw.findlaw.com/us-supreme-court/14-181.html': ['GOBEILLE, CHAIR OF THE VERMONT GREEN MOUNTAIN CARE BOARD . LIBERTY MUTUAL INSURANCE CO.'], 'http://caselaw.findlaw.com/us-supreme-court/15-7786.html': ['CHRISTOPHER EUGENE BROOKS v. ALABAMA'], 'http://caselaw.findlaw.com/us-supreme-court/14-1375.html': ['CRST VAN EXPEDITED, INC. . EQUAL EMPLOYMENT OPPORTUNITY COMMISSION'], 'http://caselaw.findlaw.com/us-supreme-court/14-1095.html': ['MUSACCHIO . UNITED STATES'], 'http://caselaw.findlaw.com/us-supreme-court/14-857.html': ['CAMPBELL-EWALD CO. . GOMEZ'], 'http://caselaw.findlaw.com/us-supreme-court/15-861.html': ['UNITED STUDENT AID

In [135]:
supcourt = pd.DataFrame(souptitles,soupurls).reset_index()
supcourt.columns = [["caseurl", "casetitle"]]

def cleanerup(text):
    text = text.replace("<EM>", "")
    text = text.replace("<SPAN STYLE=", "")
    text = text.replace("</EM>", " ")
    return text
supcourt["casetitle"] = supcourt.casetitle.apply(cleanerup)
supcourt["years"] = pd.Series(yearlist)

In [5]:
supcourt.to_pickle("supcourt_yearlist.pickle")

In [9]:
supcourt.ix[20607, "caseurl"]

'http://caselaw.findlaw.com/us-supreme-court/15-338.html'

In [10]:
supcourt.casetitle.to_csv("casetitles.csv") # this was just for creating a stoplist for prepocessing 

In [11]:
supcourt.shape

(23268, 3)